In [1]:
%pylab
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate

# %matplotlib qt

In [3]:
n_core = 10
N = 256/2
print(list(linspace(1,N+1-N//n_core,n_core,dtype='int')))
print(list(linspace(N//n_core,N,n_core,dtype='int')))

n_dt = 5000

# integer          , parameter :: ia(ni)=[1,257,513,769]
# integer          , parameter :: ib(ni)=[256,512,768,1024]

[1, 13, 26, 39, 52, 65, 78, 91, 104, 117]
[12, 24, 37, 50, 63, 76, 89, 102, 115, 128]


In [4]:
%run functions
matplotlib.rcParams.update({'font.size': 25})

In [5]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [8]:
file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20221006' # 20220610
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220930' # 20220610
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220512'
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220510'

all_subdir = []
all_Temp = []
all_trjN = []
all_trj1 = []
all_xva_3DHarmo = []
all_xva_start = []

# selection of all directories where
# there are data files
for i,j in enumerate(os.walk(file_path)):
#     print(i,j)
    if 'Try' in j[0]:
        all_subdir.append(j[0])
    # selection of all data files
    # interesting me
    for k,l in enumerate(j[2]):
#         print(k,l)
        if 'TrjN' in l:
            all_trjN.append(j[0]+'/'+l)
        if 'Trj1' in l:
            all_trj1.append(j[0]+'/'+l)
            #print(l)
        if 'Temp_SimuType0' in l and '.dat' in l:
            all_Temp.append(j[0]+'/'+l)
        if 'xva_3D' in l:
            all_xva_3DHarmo.append(j[0]+'/'+l)
        if 'xva_start' in l:
            all_xva_start.append(j[0]+'/'+l)
all_subdir.sort()
all_Temp.sort()
all_xva_3DHarmo.sort()
all_xva_start.sort()
all_trjN.sort()
# for i in range(len(all_subdir)):
#     print(all_subdir[i],'\n',all_Temp[i],'\n',all_trjN[i],'\n')
print(all_subdir)

['/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC05_RF05/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC06_RF06/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC07_RF07/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC08_RF08/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC09_RF09/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC10_RF10/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC11_RF11/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC12_RF12/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC13_RF13/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC19_RF19/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC20_RF20/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC21_RF21/Try00', '/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC22_RF22/Try00'

In [7]:
for j,k in enumerate(all_subdir):
    str_N  = all_Temp[j].find('_N')
    cond_N = all_Temp[j][str_N+3:str_N+7]
    str_v  = all_subdir[j].find('DC')
    cond   = all_Temp[j][str_v:str_v+9]
    print(cond)
    outfile = 'Time_and_temp_RFHEAT_N'+ str(cond_N) + '_' + str(cond)
    
    data = loadtxt(all_Temp[j])
    time_s = data[:,0]
    T_CM = data[:,1:4]
    T_aux = data[:,4:]
    T_aux_avg = mean(T_aux,axis=1)
    r2_v2 = load_xv2rlim(all_trjN[j])
    #r2    = r2_v2[0:3,:]
    #v2    = r2_v2[3:6,:]
    #rlim  = r2_v2[6:9,:]
    
    np.savez( outfile, time=time_s, temp=T_aux_avg, r2_v2_rlim=r2_v2) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)
    print(outfile) # , r2_v2_rlim=r2_v2

DC13_RF13
[36000000]


KeyboardInterrupt: 

In [9]:
def load_xyz_init_bin_DP(str_load):
    fid = open(str_load+'.bin',  'rb')   
    a    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    N_ions = aux[0]   

    b    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    j_save = aux[0]     

    d    = fromfile(fid,  int32, 1)        # Read record start tag   
    dt   = fromfile(fid,  float64, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag
    
    e    = fromfile(fid,  int32,   1   )        # Read record start tag   
    iRF  = fromfile(fid,  int32, 3)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag   

    f    = fromfile(fid,  int32,   1   )        # Read record start tag 
    print(f)
    save_xva = fromfile(fid,  float64, 12*N_ions)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag
    print(junk)

    fid.close
    save_xva = reshape(save_xva, (12,N_ions), order='F')

    return save_xva

def plot_XYZ(file_name,fig_name='2',fig_title='XYZ'):
    save_xva = load_xyz_init_bin_DP(file_name)
    r_LC = save_xva[0:3,:] 
    v_LC = save_xva[3:6,:]
    a_LC = save_xva[6:9,:]
    v_LC_avg = save_xva[9:,:]
    print(r_LC)
    fig = figure(fig_name); clf()
#     title(fig_title)
    gs = GridSpec(2,2, figure=fig)
    ax1 = fig.add_subplot(gs[0, :])
    ax2 = fig.add_subplot(gs[1, 0],aspect=1.0)
    ax3 = fig.add_subplot(gs[1, 1],aspect=1.0)
    ax1.plot(r_LC[2,:]*1e6,r_LC[0,:]*1e6,'8',color='xkcd:purplish blue')
    ax1.set_xlabel('z [µm]')
    ax1.set_ylabel('x [µm]')
    ax1.grid()

    # subplot(212,aspect=1.0)
    ax2.plot(r_LC[2,:]*1e6,r_LC[0,:]*1e6,'8',color='xkcd:purplish blue')
    ax2.set_xlabel('z [µm]')
    ax2.set_ylabel('x [µm]')
    ax2.grid()
    
    ax3.plot(r_LC[0,:]*1e6,r_LC[1,:]*1e6,'8',color='xkcd:purplish blue')
    ax3.set_xlabel('x [µm]')
    ax3.set_ylabel('y [µm]')
    ax3.grid()
    
    tight_layout()

def load_xva_init_bin_Lan(str_load, flag_print):      
    fid = open(str_load+'.bin',  'rb')   
    a    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    N_ions = aux[0]   

    b    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    j_save = aux[0]     

    d    = fromfile(fid,  int32, 1)        # Read record start tag   
    dt   = fromfile(fid,  float64, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag
    
    e    = fromfile(fid,  int32,   1   )        # Read record start tag   
    iRF  = fromfile(fid,  int32, 3)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag   

    f    = fromfile(fid,  int32,   1   )        # Read record start tag 
    print(f)
    save_xva = fromfile(fid,  float64, 12*N_ions)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag
    print(junk)

    fid.close
    save_xva = reshape(save_xva, (12,N_ions), order='F')

    return N_ions, j_save, dt, iRF, save_xva

In [10]:
for k,l in enumerate(all_subdir):
    print(k,l)

0 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC05_RF05/Try00
1 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC06_RF06/Try00
2 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC07_RF07/Try00
3 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC08_RF08/Try00
4 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC09_RF09/Try00
5 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC10_RF10/Try00
6 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC11_RF11/Try00
7 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC12_RF12/Try00
8 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC13_RF13/Try00
9 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC19_RF19/Try00
10 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC20_RF20/Try00
11 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC21_RF21/Try00
12 /home/adrien/RemoteFS/Rivendel/Simulations/20221006/N1024/DC22_RF22/Try00
13 /home/

In [181]:
to_plot = 11
address = all_subdir[to_plot]
print(address)
save_xva = load_xyz_init_bin_DP(all_xva_start[to_plot].strip('.bin'))
r2_v2 = load_xv2rlim(all_trjN[to_plot])
print(all_trjN[to_plot])
# my_file = 'xva_3D_Harmo_N1024_T01000uK_F0.50D-19Kg_s_5'
# print(address+'/'+my_file)
# N_ions, j_save, dt, iRF, save_xva = load_xva_init_bin_Lan(address+'/'+my_file,1)

# print(N_ions, j_save, dt, iRF, save_xva)
# figure('XYZ_Lan',clear=True)
# ax = subplot(111)
# ax.plot(save_xva[0],save_xva[1],marker='.',ls='')
# ax.grid()
# ax.set_aspect('equal')

plot_XYZ(all_xva_start[to_plot].strip('.bin'),
         fig_name='2',fig_title='XYZ')

/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N256/DC05_RF05/Try00
[24576]
[24576]
[36000000]
[36000000]
/home/adrien/RemoteFS/Rivendel/Simulations/20221006/N256/DC05_RF05/Try00/xva_SimuType0_N00256_Vrf0055_Udc0.1041D+02V_nt1000_TrjN.bin
[24576]
[24576]
[[-5.20007270e-05 -3.88117609e-05  2.73763410e-05 -5.06988488e-07
   5.41729406e-06 -2.94024247e-05 -3.15080708e-05  5.15122317e-05
   1.67525338e-05 -5.08459111e-06  2.39433215e-05 -5.29374490e-05
  -4.08781267e-05 -3.45857365e-05 -5.23063364e-05 -6.54807214e-06
  -1.22532216e-05 -2.48488641e-05 -2.13186705e-05  5.58496484e-05
  -2.70298768e-05 -5.62049170e-06  1.38179833e-05 -1.40889779e-05
  -9.77927348e-06  4.42215970e-05  1.73038491e-05 -3.79663613e-05
  -1.80347198e-05 -2.18658654e-05 -6.04641688e-06  5.06647593e-05
  -3.94548939e-05 -4.19102493e-05 -2.34805917e-06 -1.14594163e-05
  -3.16848241e-05  1.83565231e-05 -4.63527643e-05 -1.97874188e-05
  -8.13144341e-06  4.20372076e-06  2.67939279e-05  3.57160958e-05
   8.14085408e

In [183]:
print(max(save_xva[0])*1e6,max(save_xva[1])*1e6,max(sqrt(save_xva[0]**2+save_xva[1]**2))*1e6,max(abs(save_xva[2]))*1e6)
print(max(save_xva[0])/max(save_xva[2]))
print(max(save_xva[1])/max(save_xva[2]))
print('')
print(r2_v2[1]/r2_v2[0])
# print(r2_v2[1]/r2_v2[2])

55.849648388842354 55.6740586282741 56.05780991841506 51.35463408834121
1.0950058709493284
1.0915632025664916

[0.06554856 0.99635363 1.04542395 ... 0.9967976  0.99525332 0.99955557]


In [180]:
figure('histogramme',clear='True')
subplot(211)
plt.hist(sqrt(save_xva[0]**2+save_xva[1]**2)*1e3,edgecolor='k', bins=np.linspace(0,1e-1,100)) # np.linspace(0,100,20)
grid()
xlabel('r [mm]')
subplot(212)
plt.hist(abs(save_xva)[2]*1e3,edgecolor='k', bins=np.linspace(-0,1e-1,100)) # np.linspace(0,100,20)
grid()
xlabel('z [mm]')
tight_layout()

In [20]:
j = 18 # all_subdir[3]
str_N  = all_Temp[j].find('_N')
cond_N = all_Temp[j][str_N+3:str_N+7]
str_v  = all_subdir[j].find('DC')
cond   = all_Temp[j][str_v:str_v+9]
print(cond)
outfile = 'Time_and_temp_RFHEAT_N'+ str(cond_N) + '_' + str(cond)

data = loadtxt(all_Temp[j])
time_s = data[:,0]
T_CM = data[:,1:4]
T_aux = data[:,4:]
T_aux_avg = mean(T_aux,axis=1)
r2_v2 = load_xv2rlim(all_trjN[j])
#r2    = r2_v2[0:3,:]
#v2    = r2_v2[3:6,:]
#rlim  = r2_v2[6:9,:]

np.savez( outfile, time=time_s, temp=T_aux_avg, r2_v2_rlim=r2_v2) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)
print(outfile) # , r2_v2_rlim=r2_v2

DC28_RF28
[36000000]
[36000000]
Time_and_temp_RFHEAT_N1024_DC28_RF28


# With extended

In [335]:
file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220930' # 20220610
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220512'
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220510'

all_subdir = []
all_Temp = []
all_Temp_extended = []
all_trjN = []
all_trjN_extended = []
all_trj1 = []
all_xva_3DHarmo = []
all_xva_start = []

# selection of all directories where
# there are data files
for i,j in enumerate(os.walk(file_path)):
#     print(i)
    if 'Try' in j[0]:
        all_subdir.append(j[0])
    # selection of all data files
    # interesting me
    for k,l in enumerate(j[2]):
        if 'TrjN' in l:
            all_trjN.append(j[0]+'/'+l)
            all_trjN_extended.append('')
        if 'TrjN' in l and 'extended' in l:
            all_trjN_extended[i-2] = f'{j[0]}/{l}'
        if 'Trj1' in l:
            all_trj1.append(j[0]+'/'+l)
            #print(l)
        if 'Temp_SimuType0' in l and '.dat' in l and not 'extended' in l:
            all_Temp.append(j[0]+'/'+l)
            all_Temp_extended.append('')
        if 'Temp_SimuType0' in l and '.dat' in l and 'extended' in l:
            all_Temp_extended[i-2] = f'{j[0]}/{l}'
        if 'xva_3D' in l:
            all_xva_3DHarmo.append(j[0]+'/'+l)
        if 'xva_start' in l:
            all_xva_start.append(j[0]+'/'+l)
# all_subdir.sort()
# all_Temp.sort()
# all_Temp_extended.sort()
# all_xva_3DHarmo.sort()
# all_xva_start.sort()
# all_trjN.sort()
# for i in range(len(all_subdir)):
#     print(all_subdir[i],'\n',all_Temp[i],'\n',all_trjN[i],'\n')

In [322]:
for j,k in enumerate(all_subdir):
    str_N  = all_Temp[j].find('_N')
    cond_N = all_Temp[j][str_N+3:str_N+7]
    str_v  = all_subdir[j].find('DC')
    cond   = all_Temp[j][str_v:str_v+9]
    print(cond)
    outfile = 'xxxxTime_and_temp_RFHEAT_N'+ str(cond_N) + '_' + str(cond)
    
    data = loadtxt(all_Temp[j])
    try:
        data_extended = loadtxt(all_Temp_extended[j])
        time_s = hstack((data[:,0],data_extended[:,0]))
        T_CM = hstack((data[:,1:4],data_extended[:,1:4]))
        T_aux = hstack((data[:,4:],data_extended[:,4:]))
        T_aux_avg = mean(T_aux,axis=1)
        r2_v2 = load_xv2rlim(all_trjN[j])
        r2_v2_extended = load_xv2rlim(all_trjN_extended[j])
        r2_v2 = hstack((r2_v2 ,r2_v2_extended))
        print('coucou')
    except:
        time_s = data[:,0]
        T_CM = data[:,1:4]
        T_aux = data[:,4:]
        T_aux_avg = mean(T_aux,axis=1)
        r2_v2 = load_xv2rlim(all_trjN[j])
        print('pas coucou')
        #r2    = r2_v2[0:3,:]
        #v2    = r2_v2[3:6,:]
        #rlim  = r2_v2[6:9,:]
    
    np.savez( outfile, time=time_s, temp=T_aux_avg, r2_v2_rlim=r2_v2) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)
    print(outfile) # , r2_v2_rlim=r2_v2

DC04_RF04
[101524680]
[101524680]
xxxxTime_and_temp_RFHEAT_N0512_DC04_RF04
DC04_RF04
[101524680]
[101524680]
xxxxTime_and_temp_RFHEAT_N0512_DC04_RF04
DC04_RF04
[50762304]
[50762304]
xxxxTime_and_temp_RFHEAT_N0512_DC04_RF04


In [325]:
all_subdir[1]

'/home/adrien/RemoteFS/Rivendel/Simulations/20220930/DC04_RF04/Try01'

In [438]:
j = 1
print(all_subdir[j])
str_N  = all_Temp[j].find('_N')
cond_N = all_Temp[j][str_N+3:str_N+7]
str_v  = all_subdir[j].find('DC')
cond   = all_Temp[j][str_v:str_v+9]
print(cond)
outfile = 'xxxxstackTime_and_temp_RFHEAT_N'+ str(cond_N) + '_' + str(cond)

data = loadtxt(all_Temp[j])
try:
    data_extended = loadtxt(all_Temp_extended[j])
    time_s = hstack((data[:,0],array(data_extended[:,0])+max(data[:,0])-min(data[:,0])))
    T_CM = vstack((data[:,1:4],data_extended[:,1:4]))
    T_aux = vstack((data[:,4:],data_extended[:,4:]))
    T_aux_avg = mean(T_aux,axis=1)
    r2_v2 = load_xv2rlim(all_trjN[j])
    r2_v2_extended = load_xv2rlim(all_trjN_extended[j])
    r2_v2 = hstack((r2_v2 ,r2_v2_extended))
    print('coucou')
except:
    time_s = data[:,0]
    T_CM = data[:,1:4]
    T_aux = data[:,4:]
    T_aux_avg = mean(T_aux,axis=1)
    r2_v2 = load_xv2rlim(all_trjN[j])
    print('pas coucou')
    #r2    = r2_v2[0:3,:]
    #v2    = r2_v2[3:6,:]
    #rlim  = r2_v2[6:9,:]

np.savez( outfile, time=time_s, temp=T_aux_avg, r2_v2_rlim=r2_v2) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)
print(outfile) # , r2_v2_rlim=r2_v2
print('fini')

/home/adrien/RemoteFS/Rivendel/Simulations/20220930/DC04_RF04/Try01
DC04_RF04
[101524680]
[101524680]
[50762304]
[50762304]
coucou
xxxxstackTime_and_temp_RFHEAT_N0512_DC04_RF04
fini


array([0.00100062, 0.00100112, 0.00100162, ..., 0.35351512, 0.35351562,
       0.35351612])

In [430]:
loadtxt(all_Temp[1])[0],time_s_extended

(array([1.00062450e-03, 1.03382074e-05, 1.37773710e-05, 7.12058867e-05,
        6.20725325e-03, 1.01597441e-02, 9.23096771e-03]),
 array([1.00062450e-03, 1.03382074e-05, 1.37773710e-05, 7.12058867e-05,
        6.20725325e-03, 1.01597441e-02, 9.23096771e-03]))

In [416]:
T_aux = vstack((data[:,4:],data_extended[:,4:]))
shape(T_aux)
shape(mean(T_aux,axis=1))

(2115097,)